In [1]:
#Import Dependencies
import requests
import pandas as pd
from character_lists import char_names
import csv
import json

In [2]:
#Read in csv and character list
j = "comics.csv"
comics = pd.read_csv(j)
comics.drop(comics.columns[0], axis=1, inplace=True)
comics.head()

FileNotFoundError: [Errno 2] File comics.csv does not exist: 'comics.csv'

In [ ]:
#Convert strings in df columns to series and delete old columns
#Start with dates
list_of_dates=[]
comic_date = comics['Date']
for indiv_date in comic_date:
    date_list = eval(indiv_date)
    list_of_dates.append(date_list)
date_series = pd.Series(list_of_dates)
comics['Date Series'] = date_series
#Repeat with characters
list_of_chars=[]
comic_char = comics['Characters']
for indiv_char in comic_char:
    char_list = eval(indiv_char)
    list_of_chars.append(char_list)
char_series = pd.Series(list_of_chars)
comics['Character Series'] = char_series
#Repeat with creators
list_of_creates=[]
comic_creates = comics['Creators']
for indiv_create in comic_creates:
    create_list = eval(indiv_create)
    list_of_creates.append(create_list)
create_series = pd.Series(list_of_creates)
comics['Creator Series'] = create_series
comics.drop(comics.columns[[2, 3, 4]], axis=1, inplace=True)
comics.head()

In [ ]:
#Extract Earliest Date and replace
list_of_dates = []
for i in comics['Date Series']:
    j = i[0]['date']
    k = j[0:4]
    k= int(k)
    if  (k > 1900) & (k < 2030):
        list_of_dates.append(k)
    else:
        list_of_dates.append(0)
date_series = pd.Series(list_of_dates)
comics['Date'] = date_series
print(date_series)
comics.drop(comics.columns[2], axis=1, inplace=True)
comics.tail()

In [ ]:
#flatten df by making rows for creators (but preserve one row so flattening on characters can occur)
flat_comics_df = pd.DataFrame()
flat_id_list = []
flat_title_list = []
flat_date_list = []
flat_creators_list = []
flat_characters_list = []
for i, v in comics.iterrows(): 
    try: #try looping through creators, and nested looping through characters
                creators = v[3]
                for j in creators:
                    flat_creator = j['name']
                    try: #adds new row for each character that includes current creator
                        characters = v[2]
                        for k in characters:
                            flat_char = k['name']
                            flat_characters_list.append(flat_char)
                            flat_creators_list.append(flat_creator)
                            flat_id_list.append(v[0])
                            flat_title_list.append(v[1])
                            flat_date_list.append(v[4])
                    except:
                            flat_characters_list.append("")
                            flat_creators_list.append(flat_creator)
                            flat_id_list.append(v[0])
                            flat_title_list.append(v[1])
                            flat_date_list.append(v[4])                   
    except:
                    try:
                        characters = v[2]
                        for k in characters:
                            flat_char = k['name']
                            flat_characters_list.append(flat_char)
                            flat_creators_list.append("")
                            flat_id_list.append(v[0])
                            flat_title_list.append(v[1])
                            flat_date_list.append(v[4])
                    except:
                        pass
dict = {"ID": flat_id_list, "Title": flat_title_list, "Date": flat_date_list, "Creator": flat_creators_list, "Character": flat_characters_list}
cdcc_df = pd.DataFrame(dict)



In [ ]:
for title in cdcc_df['Title']:
    title_slice = title[-5:-1]
    try:
      int_slice = int(title_slice)
      if  (int_slice > 1900) & (int_slice < 2030):
        cdcc_df.loc[cdcc_df['Title'] == title]['Date'] = int_slice
        print(cdcc_df.loc[cdcc_df['Title'] == title]['Date'])
    except:
        pass


In [ ]:
cdcc_df.head()

In [ ]:
list_of_chars=[]
comic_char = comics['Characters']
for indiv_char in comic_char:
    char_list = eval(indiv_char)
    list_of_chars.append(char_list)
type(list_of_chars)

In [ ]:
#Question: what's the distribution of appearances of Marvel characters?
char_counts = {}
for i in list_of_chars:
    for j in i:
        char_key = j['name']
        if char_key in char_counts.keys():
            char_counts[char_key] = char_counts[char_key]+1
        else:
            char_counts[char_key] = 1
print(char_counts)

In [ ]:
cdcc_df.head()
cdcc_df.to_csv("ccdc_df.csv")

In [ ]:
#See the numbers of appearances of Marvel Characters
char_counts = cdcc_df.groupby('Character')['ID'].count()
print(char_counts)
# import itertools
# char_sort = dict(sorted(char_counts.items(), key=lambda item: item[1], reverse = True))
# N = 500
# chars_500 = dict(itertools.islice(char_sort.items(), N))
# print(chars_500)

In [ ]:
#See number of unique creators for characters
char_creators = cdcc_df.groupby('Character')['Creator'].nunique()
char_creators.sort_values(ascending=False, inplace=True)
print(char_creators)

In [ ]:
#See the number of unique characters for creators
creator_chars = cdcc_df.groupby('Creator')['Character'].nunique()
creator_chars.sort_values(ascending=False, inplace = True)
print(creator_chars)

In [ ]:
#DataFrame of Top 500 Marvel Chars with Characters as Index
char_500_df = pd.DataFrame.from_dict(chars_500, orient='index')
char_500_df.head(1400)

In [ ]:
list_of_creates=[]
comic_creates = comics['Creators']
for indiv_create in comic_creates:
    create_list = eval(indiv_create)
    list_of_creates.append(create_list)
print(list_of_creates)

In [ ]:
create_counts = {}
for i in list_of_creates:
    for j in i:
        create_key = j['name']
        if create_key in create_counts.keys():
            create_counts[create_key] = create_counts[create_key]+1
        else:
            create_counts[create_key] = 1
print(create_counts)

In [ ]:
#Find top 500 Creators
creator_sort = dict(sorted(create_counts.items(), key=lambda item: item[1], reverse = True))
N = 500
creators_500 = dict(itertools.islice(creator_sort.items(), N))
print(creators_500)


In [ ]:
#DataFrame of Top 500 Marvel Creators with Creators as Index
creators_500_df = pd.DataFrame.from_dict(creators_500, orient='index')
creators_500_df.head(40)

In [ ]:
#What characters are the top 500 Creators associated with
cc_df = pd.DataFrame()
for creator in creators_500:
    creator_df = comics.loc[comics["Creators"] ]
creator_df.head()
# for row in comics.iterrows():
#     row_creator = row[5]
#     row_character = row[4]
#     for creators in row_creator:
#         for creator, count in creators_500:
#             if creator==creators:
#                 for character in row_character:
#                     cc_df[creator][character]= cc_df[creator][character]+1

# cc_df.head()


In [ ]:
#Question: are Marvel characters old (i.e., were they created long ago)?

In [ ]:
#Question: did Marvel CU characters tend to appear together?

In [ ]:
#Question: what's the distribution over time of appearances of Marvel characters?

In [ ]:
#What distribution of creators is each character associated with?

In [ ]:
#Which creators were most associated with the MCU

In [ ]:
#When did the creators most associated with the MCU create?

In [ ]:
#What does this suggest about the MCU?

In [ ]:
#Before or after Avengers? [categorical]

In [ ]:
#Prominence of before or after Avengers